<a href="https://colab.research.google.com/github/nathUjjal/Multi-Modal-Fake-News-Detection/blob/main/notebooks/imageBasedDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 22.6 MB/s eta 0:00:00


In [5]:
from transformers import pipeline,BlipProcessor,BlipForConditionalGeneration
import torch
from io import BytesIO
import requests
from PIL import Image
import numpy as np
import easyocr

In [14]:
model=BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
processor=BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

In [15]:
classifier=pipeline('text-classification',model="facebook/bart-large-mnli")

Device set to use cpu


In [16]:
def generate_caption(image):
    inputs=processor(image,return_tensors="pt")
    out=model.generate(**inputs)
    caption=processor.decode(out[0],skip_special_tokens=True)
    return caption

In [6]:
def extract_test(image):
  ocr=easyocr.Reader(['en'])
  extracted_text=ocr.readtext(image,details=0)
  return extracted_text

In [11]:
def retrieve_evidence(claim,max_sentence=2):
  pass

In [10]:
def verify_claim(claim,evidence):
  results=classifier({'text':evidence,'text_pair':claim},truncation=True)
  label=results[0].label
  score=results[0].score
  return label,score

In [13]:
def image_based_verdict(label,score):
    if label=='contradiction' and score > 0.7 :
      return {'img_status':"FAKE",'img_score':score}
    elif label=='entailment' and score > 0.7 :
      return {'img_status':"REAL",'img_score':score}
    else:
      return {'img_status':"UNCERTAIN",'img_score':score}